# Guide to use the classifiers of the Quantum_Classifiers_BSQ_201 repository

## The utils folder

The utils of the repository folder can be very useful for running the quantum classifiers. Its functions will be described here.

### Quantum embeddings

These are the 3 embeddings coded in pennylane that are given in this repository. Here is how to import them.

In [15]:
from utils.quantum_embeddings import angle_embedding,iqp_embedding,amplitude_embedding

The IQP embedding is only specified for a number of qubits equal to the number of features. It takes the feature vector as an input.

The amplitude embedding will automatically create a circuit with the superior or equal power of two of the number of features. It takes the feature vector as an input.

The angle embedding takes the number of qubits and rotation gates as a parameter. To use it in the next algorithms, a wrapper like below must be created so that the embedding function only takes the feature vector as a parameter. The choices of the rotation gates are "X", "Y" and "Z". 

In [16]:
num_qubits=4
rotation="X"
def angle_embedding(a):
        return angle_embedding(
            a, num_qubits=num_qubits, rotation=rotation
        )

If an embedding is called by itself, a pennylane circuit is created. These signatures of embeddings will be the ones used in the classifier classes.

### Ansatz

The utils folder also provides a working ansatz for the VQC algorithm with the *HTRU_2* dataset. To call it, just run this line of code:

In [17]:
from utils.quantum_ansatz import ansatz_circuit

There is also a randomized ansatz using Pennylane's random layers. To use it in the classifiers, like the angle embedding, a wrapper needs to be created. For example:

In [18]:
from utils.quantum_ansatz import ansatz_random_layer

def ansatz(params):
    return ansatz_random_layer(params,num_qubits=8,num_params_per_qubits=4)

### Error functions

Multiple error functions are also provided by the utils folder. They can be used as the function to optimize (cost_function) in the VQC and QCNN algorithms. To know more about those functions, read the article in the repository. Here is the complete list.

In [19]:
from utils.error_functions import (
    mean_square_error, 
    normalized_mean_square_error,
    normalized_root_mean_square_error,
)

There are also multiple functions to compare the different accuracies of the classifier. They can not be used in the VQC and QCNN algorithms since they do not satisfy the conditions for an error function. These conditions will be described later.

In [20]:
from utils.error_functions import (
    accuracy,
    recall,
    specifity,
    precision,
    negative_prediction_value,
    balanced_accuracy,
    geometric_mean,
    informedness,
)

### Other functions

Three other useful functions can be found in the utils folder. 

The *get_feature_vectors_and_labels* function allows to read datasets from a file. The path, file name and extensions are all given. Also, the number of rows to skip to access the data is set by default at zero.

*get_good_distribution_of_labels* gives an even distribution of the feature vectors of the dataset of the label 1 and -1. The feature vectors and their respective labels need to be given to the function.

Finally, *normalize_feature_vectors* normalizes each feature of the feature vectors so that its minimum value is -1 and its maximum is 1. It is useful to do so to make sure that each parameter starts with the same importance and to make sure that the rotation gate actually means something with each vector (since they are 2pi periodical). It only takes the feature vectors for a parameter.

They are imported from the line below. Their usage will be shown directly in the next section of the tutorial.

In [21]:
from utils.utils import (
    get_feature_vectors_and_labels, 
    get_good_distribution_of_labels,
    normalize_feature_vectors,
)

# First step: Load the data

The first step to run the algorithms is to load a dataset to classify. To do so, we will use the three functions described in the previous section. For this tutorial, we will use the *HTRU_2* dataset based on pulsar detection. It is available under the dataset folder in the repository. For the VQC and QCNN algorithms, the labels must be -1 or 1. The function *get_good_distribution_of_labels* only works if those labels are respected. So, in the code below, there will be an example of changing the 0 labels in the datasets into -1.

Here is how to load a dataset, get a significant sample (not only zeros or one to make sure that the classifiers work correctly) and normalize the features with the utils function described earlier. The data can be loaded differently as long as there is a feature vectors matrix and a label array or 1 and -1. These types of inputs will be the ones that will work across all of the algorithms. Only the quantum kernel classifier does not need labels of -1 and 1. They just need to be 2 differents integers.

In [22]:
from utils.utils import (
    get_feature_vectors_and_labels, 
    get_good_distribution_of_labels,
    normalize_feature_vectors,
)

#Load the dataset
feature_vectors, labels = get_feature_vectors_and_labels(
        "HTRU_2", extension="csv", path="datasets/", rows_to_skip=0
    )

#Change the 0 labels into -1
labels=(2*labels)-1
# Get a good sample of the dataset since it is too big
feature_vectors, labels = get_good_distribution_of_labels(
    feature_vectors, labels, 50
)
# Normalize the feature vectors
feature_vectors = normalize_feature_vectors(feature_vectors)

# Quantum classifiers
## Note

The use of the three different quantum classifiers is very similar. The quantum kernel algorithm will be described in detail and will be referenced for the other methods to describe their functioning.

## The quantum kernel classifier

To use the quantum kernel classifier, the *Quantum_Kernel_Classification* class from the *kernel_method.py* file needs to be called.

First, run this line to import the class.

In [23]:
from kernel_method import Quantum_Kernel_Classification

Then, the object needs an embedding function and the number of qubits of that embedding. The embedding needs to be a function with the same number of qubits given to the class. It must create the Pennylane circuit that encodes the feature vector. It can not be a QNODE.

Let's create the object using the amplitude embedding function from the utils file.

In [24]:
import utils.quantum_embeddings
num_qubits=3

kernel_qml = Quantum_Kernel_Classification(
        utils.quantum_embeddings.amplitude_embedding, num_qubits
    )

An object ready to run the quantum kernel classification is created.

To run the algorithm  with a dataset, call the run function. It needs the feature vectors and their respective labels which are two integers.

The optional argument, training ratio, is fixing the ratio between the number of training vectors to be used from all of them given.

Let's call the run function.

In [25]:
training_ratio=0.8

score, predictions = kernel_qml.run(
        feature_vectors, labels, training_ratio=training_ratio
    )

The result of this algorithm is in two parts.

The score return gives the number of labels that the classifier correctly predicted.

The prediction return is the array of all the predicted labels.

Those results can then be printed to evaluate the precision of the algorithm.

In [26]:
training_period = int(len(labels) * training_ratio)

print("The score of the kernel: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the kernel:  0.85
The predictions of the labels:  [-1 -1 -1  1  1  1  1  1  1 -1  1  1 -1  1  1  1 -1  1  1 -1]
The true value of the labels:  [-1  1 -1  1  1 -1  1  1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1]


## The quantum variational algorithm

To use the quantum kernel classifier, the *VQC_Solver* class from the *vqc_method.py* file needs to be called.

First, run this line to import the class.

In [27]:
from vqc_method import VQC_Solver

Then, the object needs an embedding function, the ansatz circuit, the number of parameters in the ansatz and the number of qubits of the two circuits.

Like the embedding function described in the kernel classifier, the ansatz callable is a function creating a pennylane circuit that is to be optimized. It can not be a QNODE. It must also have the same number or less of qubits than the one given to the class. It also has the same number or less of parameters than the number given to the class.
Let's create the object using the amplitude embedding function.

In [28]:
import utils.quantum_embeddings
import utils.quantum_ansatz

num_qubits=8
num_params = 12

vqc = VQC_Solver(
    utils.quantum_embeddings.amplitude_embedding,
    utils.quantum_ansatz.ansatz_circuit,
    num_params,
    num_qubits,
)

An object ready to run the VQC (variational quantum classifier) is created.

To run the algorithm  with a dataset, the same procedure needs to be filled as the quantum kernel method.
However, the run function also needs the optimizer, the error function to be minimized and the classifier function. Also, the labels must be -1 or 1.

The optimizer given needs to only have two parameters: the cost_function and its parameters to optimize. The result of the optimizer also needs an *x* attribute to give the optimized parameters. In this tutorial, we will create a wrapper optimization function that calls the COBYLA method from scipy's minimizer.

The error function has also two parameters (the predicted labels and their real value). It is the error to be minimized by the optimizer. Multiple error functions are given in the utils file as specified earlier. The error function must directly use the expval values of the feature vectors in the circuit for the calculation. It can not use a copy of those results. By default, the mean square error function of the utils folder is used.

So, let's use the run function.

In [29]:
from scipy.optimize import minimize
from utils.error_functions import mean_square_error
training_ratio=0.8

def minimisation(cost_function, params):
        result=minimize(
            cost_function,
            params,
            method="COBYLA",
            options={"maxiter": 25},
        )
        return result.x

score, predictions = vqc.run(
    feature_vectors, labels, minimisation,error_function=mean_square_error, training_ratio=training_ratio
)

The result of this algorithm is the same as the quantum kernel algorithm. Let's print the results.

In [30]:
training_period = int(len(labels) * training_ratio)

print("The score of the VQC: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the VQC:  0.85
The predictions of the labels:  [-1 -1 -1 -1  1  1  1  1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1]
The true value of the labels:  [-1  1 -1  1  1 -1  1  1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1]


## The quantum convolutional neural network algorithm

To use the quantum kernel classifier, the *QCNN_Solver* class from the *qcnn_method.py* file needs to be called.

First, run this line to import the class.

In [31]:
from qcnn_method import QCNN_Solver

Then, the object needs an embedding function and the number of qubits of this circuit.
Let's create the object using the amplitude embedding function.

In [32]:
import utils.quantum_embeddings

num_qubits=8
def angle(feature_vectors):
    return utils.quantum_embeddings.angle_embedding(feature_vectors,num_qubits,rotation="Y")

qcnn = QCNN_Solver(angle, num_qubits)

An object ready to run the QCNN (quantum convolutional neural network) is created.

To run the algorithm  with a dataset, the same procedure as the VQC method needs to be followed (the same condition for the optimizer, error function, labels  and classifier function).

However, there are two run function.
Let's first use the normal run function. It uses all of the training parameters at each iteration of the optimization.


So, let's use the run function.

### Run function

In [33]:
from utils.error_functions import mean_square_error
from pennylane.optimize import NesterovMomentumOptimizer
training_ratio=0.8

def minimisation(cost_function, params):
    result=NesterovMomentumOptimizer(stepsize=0.1)
    new_params=params
    for _ in range(30):
        new_params=result.step(cost_function,new_params)
    return new_params    

score, predictions = qcnn.run(
    feature_vectors,
    labels,
    minimisation,
    error_function=mean_square_error,
    training_ratio=training_ratio,
)

The result of this algorithm is the same as the quantum kernel algorithm. Let's print the results.

In [34]:
training_period = int(len(labels) * training_ratio)

print("The score of the normal QCNN: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the normal QCNN:  0.65
The predictions of the labels:  [-1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1  1  1 -1]
The true value of the labels:  [-1  1 -1  1  1 -1  1  1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1]


### run_batched function

The *run_batched* function uses the same parameters as the normal run function. However, it also has a *num_batches* parameter. Indeed, this function uses partitions of the training parameters to optimize the parametrized circuit. Instead of getting the results of all of the training vectors for one optimization of the the optimizer, only a few amount of the data is used.

Each batch of the data are used once so that the training vectors are used once. We clearly have a complexity gain.

To use this function, the optimizer needs to run exactly *num_batches iterations*.

This is how to call the run_batched algorithm.

In [35]:
# Let's start over with a new object with not optimized parameter
import utils.quantum_embeddings
num_qubits=8
def angle(feature_vectors):
    return utils.quantum_embeddings.angle_embedding(feature_vectors,num_qubits,rotation="Y")

qcnn = QCNN_Solver(angle, num_qubits)

In [36]:
# To run the function:
from scipy.optimize import minimize
from utils.error_functions import mean_square_error

batches = 8
def minimisation(cost_function, params):
        result=minimize(
            cost_function,
            params,
            method="COBYLA",
            options={
                "maxiter": batches, #The number of iteration of the optimizer needs to be the same as the number of batches
            },
        )
        
        return result.x

score, predictions = qcnn.run_batched(
    feature_vectors,
    labels,
    minimisation,
    error_function=mean_square_error,
    num_batches=batches,
    training_ratio=training_ratio,
)

The result of this algorithm is the same as the quantum kernel algorithm. Let's print the results.

In [37]:
training_period = int(len(labels) * training_ratio)

print("The score of the batched QCNN: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", labels[training_period:])

The score of the batched QCNN:  0.35
The predictions of the labels:  [-1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
The true value of the labels:  [-1  1 -1  1  1 -1  1  1  1 -1  1  1 -1  1  1 -1 -1  1  1 -1]


You are now ready to use the various quantum classifiers!

# Classical classifiers

To run these two algorithms, the data must first be loaded like it was done before the quantum classifiers. Here is how to use them.

## Classical kernel method

To run this method, the *svm_run* function needs to be imported.

In [38]:
from classicial_classifiers.svm_method import svm_run

Like the quantum classifiers, this function must receive the feature vectors and their labels as parameters. The training ratio can also be given as an optional parameter. The kernel chosen can also be specified. The choice of the kernels is given by the sklearn.svm.SVC class. See its description for a list of the available kernels.

In [39]:
training_ratio=0.8
score, predictions, true_value=svm_run(feature_vectors,labels,training_ratio=training_ratio,kernel="rbf") 

We interpret the return the same way as the quantum classifiers.

In [40]:
print("The score of the SVM: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", true_value)

The score of the SVM:  0.9
The predictions of the labels:  [-1  1 -1  1  1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1  1  1 -1]
The true value of the labels:  [ 1  1 -1  1  1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1  1  1  1]


## CNN

To run this method, the *cnn_run* function needs to be imported.

In [41]:
from classicial_classifiers.cnn_method import cnn_run

Like the quantum classifiers, this function must receive the feature vectors and their labels as parameters. The training ratio can also be given as an optional parameter. The algorithm also takes as a parameter the size of the batches. 

In [42]:
training_ratio=0.8
batch_size=8

score, predictions, true_value=cnn_run(feature_vectors,labels,training_ratio=training_ratio, batch_size=batch_size) 

We interpret the return the same way as the quantum classifiers.

In [43]:
print("The score of the CNN: ", score)
print("The predictions of the labels: ", predictions)
print("The true value of the labels: ", true_value)

The score of the CNN:  0.9
The predictions of the labels:  [-1  1 -1  1  1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1  1  1 -1]
The true value of the labels:  [ 1  1 -1  1  1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1  1  1  1]


You are now ready to use the various classical classifiers!